# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 

*Note*: <br>
Because of limited GPU hours, I decided to move to Google Colab. I tried various models, and below answers are as per the BLEU scores that I obtained on colab. Once I decided my final model architecture and all the parameters, I trained a model here in Udacity workspace with that config.


*Encoder*: <br>
Tried VGG16 and ResNet50 for creating the encoder. When I compared their BLEU scores, ResNet50 performed better. <br>
VGG16 has many more parameters compared to that in ResNet50. Thus, training VGG16 took longer, and its checkpoint files were quite big (~500 MB compared to ~100MB for ResNet50).

*Decoder*:<br>
Tried 1 and 2 LSTM layers for decoder network. Decoder with 2 LSTM layers gave better BLEU scores, so decided to have 2 LSTM layers in my model.
With 2 LSTM layers, I tried dropout of 0.2 and 0.4. Both gave lower BLEU scores, so I have not added any dropout.

*batch_size*:<br>
Tried various values for batch size (e.g. 64, 128, 256, 512). If batch size is very small, you will see a lot of fluctuations in loss value in each epoch. On the other hand, if it is very large, you may get 'out of memory' errors. Also, having larger batch size means more parallelization. <br>
After comparing models trained using sizes 256 and 512, I decided to use 512 since it showed small improvement in BLEU score.

*vocab_threshold*:<br>
As per [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/pdf/1411.4555.pdf), decided to use 5.

*embed_size and hidden_size*:<br>
Tried various permutations of values for these two parameters. e.g. (256, 256), (256, 512), (512, 512), etc.
The combination (256, 1024) gave comparatively better BLEU scores, so decided to use these values.





### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

I tried below transformations. <br>
    &emsp; &emsp; RandomRotation(degrees=20) <br>
    &emsp; &emsp; GaussianBlur(kernel_size=3) <br>
    &emsp; &emsp; RandomAutocontrast() <br>

I tried them one-by-one as well as together, but every time I got comparatively lower BLEU score. Thus, I decided to not add any additional transformations.

Intuitively speaking: <br>
I am using a CNN that is already trained on a large number of variety of images. So, after removing the classification layer, the output of the last layer of this CNN should be same (or nearly same) irrespective of whether we use some transformation or not. <br>
In other words, since the encoder "encodes" the content of the input image, its output should be same even if we rotate, blur, etc. Thus, I think, applying any transformations is not likely to help much.



### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

I am using a pre-trained CNN model (ResNet50) as an encoder. Since it is already trained, its parameters should not be modified. However, I have removed the classification layer from this CNN and, in its place, have added an embedding layer. This embedding layer is completely new, so its parameters need be trained.
In decoder, I am using 2 LSTM layers and 1 FC layer. Again, these are completely new layers. Hence, their parameters should also be modified during training.

Therefore, I selected the parameters in the embedding layer and the decoder for training.


### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In SGD optimizer, you need to manually set the learning rate, whereas in Adam optimizer, you can manually set the learning rate but you are not required to. If you don't specify the learning rate, Adam optimizer automatically finds a good value.<br>
Also, as you train your model, you should reduce learning rate ("learning rate decay") after few epochs. In SGD, you need to decide yourself when you should change the learning rate and what the new learning rate should be. In Adam optimizer however this is done automatically.

Finally, Adam optimizer also has some mathematical improvements compared to SGD. For example, it uses running average of gradients while updating model parameters. This leads to lesser fluctuations in loss and also reduces the chance of model getting stuck in a local minima.

I tried both SGD and Adam optimizer. With SGD, perplexity was stuck at ~10, even when I reduced the learning rate a couple of times during training. With Adam optimizer, I managed to get perplexity <10. So, I chose Adam optimizer for training the final model.

In [1]:
### MY CODE
# if you don't execute notebook "1. Preliminaries", you need to download punkt here
!pip install nltk
import nltk
nltk.download('punkt')
### MY CODE

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 512           # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 1024          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=2)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.17s)
creating index...


  0%|          | 811/414113 [00:00<01:48, 3818.96it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:30<00:00, 4599.51it/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:00<00:00, 103626370.62it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [3]:
### MY CODE
# to resume training
encoder.load_state_dict(torch.load('./models/encoder-4.pkl'))
decoder.load_state_dict(torch.load('./models/decoder-4.pkl'))
### MY CODE

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# moved the training code inside a function to be able to execute it under active_session()
def train(start_epoch, num_epochs):
    # Open the training log file.
    f = open(log_file, 'w')

    old_time = time.time()
    response = requests.request("GET", 
                                "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                                headers={"Metadata-Flavor":"Google"})

    for epoch in range(start_epoch, num_epochs+1):

        ### MY CODE
        epoch_start = time.time()
        ### MY CODE

        for i_step in range(1, total_step+1):

            if time.time() - old_time > 60:
                old_time = time.time()
                requests.request("POST", 
                                 "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                                 headers={'Authorization': "STAR " + response.text})

            # Randomly sample a caption length, and sample indices with that length.
            indices = data_loader.dataset.get_train_indices()
            # Create and assign a batch sampler to retrieve a batch with the sampled indices.
            new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
            data_loader.batch_sampler.sampler = new_sampler

            # Obtain the batch.
            images, captions = next(iter(data_loader))

            # Move batch of images and captions to GPU if CUDA is available.
            images = images.to(device)
            captions = captions.to(device)

            # Zero the gradients.
            decoder.zero_grad()
            encoder.zero_grad()

            # Pass the inputs through the CNN-RNN model.
            features = encoder(images)
            outputs = decoder(features, captions)

            # Calculate the batch loss.
            loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))

            # Backward pass.
            loss.backward()

            # Update the parameters in the optimizer.
            optimizer.step()

            # Get training statistics.
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

            # Print training statistics (on same line).
            print('\r' + stats, end="")
            sys.stdout.flush()

            # Print training statistics to file.
            f.write(stats + '\n')
            f.flush()

            # Print training statistics (on different line).
            if i_step % print_every == 0:
                print('\r' + stats)

        # Save the weights.
        if epoch % save_every == 0:
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

        ### MY CODE
        print(f"\n\nEpoch time: {time.time()-epoch_start}\n\n")
        ### MY CODE

    # Close the training log file.
    f.close()

In [5]:
# I trained the model one epoch at a time so that I can test it after every epoch.
# Thus, I added two parameters start_epoch and num_epochs to the train() function.

# Also, to preserve the training/loss history for all the epochs, I trained each epoch in a separate notebook cell

from workspace_utils import active_session  

# pass these variables so that you can resume training
start_epoch = 1
num_epochs = 1

with active_session():
    train(start_epoch, num_epochs)

Epoch [1/1], Step [100/809], Loss: 4.6720, Perplexity: 106.9101
Epoch [1/1], Step [200/809], Loss: 3.4943, Perplexity: 32.9285
Epoch [1/1], Step [300/809], Loss: 3.2061, Perplexity: 24.6827
Epoch [1/1], Step [400/809], Loss: 3.1466, Perplexity: 23.25649
Epoch [1/1], Step [500/809], Loss: 3.1137, Perplexity: 22.5039
Epoch [1/1], Step [600/809], Loss: 2.8890, Perplexity: 17.9757
Epoch [1/1], Step [700/809], Loss: 2.7483, Perplexity: 15.6161
Epoch [1/1], Step [800/809], Loss: 2.7810, Perplexity: 16.1356
Epoch [1/1], Step [809/809], Loss: 2.6041, Perplexity: 13.5197

Epoch time: 7944.443021535873




In [6]:
from workspace_utils import active_session  

start_epoch = 2
num_epochs = 2

with active_session():
    train(start_epoch, num_epochs)

Epoch [2/2], Step [100/809], Loss: 2.4289, Perplexity: 11.3467
Epoch [2/2], Step [200/809], Loss: 2.3903, Perplexity: 10.9164
Epoch [2/2], Step [300/809], Loss: 2.6081, Perplexity: 13.5735
Epoch [2/2], Step [400/809], Loss: 2.4200, Perplexity: 11.2464
Epoch [2/2], Step [500/809], Loss: 2.2117, Perplexity: 9.13154
Epoch [2/2], Step [600/809], Loss: 2.2418, Perplexity: 9.41012
Epoch [2/2], Step [700/809], Loss: 2.1322, Perplexity: 8.43362
Epoch [2/2], Step [800/809], Loss: 2.2097, Perplexity: 9.11315
Epoch [2/2], Step [809/809], Loss: 2.5113, Perplexity: 12.3208

Epoch time: 7843.719820022583




In [7]:
from workspace_utils import active_session  

start_epoch = 3
num_epochs = 3

# start time 1:25:10
with active_session():
    train(start_epoch, num_epochs)

Epoch [3/3], Step [100/809], Loss: 2.2139, Perplexity: 9.1515
Epoch [3/3], Step [200/809], Loss: 2.0699, Perplexity: 7.92387
Epoch [3/3], Step [300/809], Loss: 2.1048, Perplexity: 8.20592
Epoch [3/3], Step [400/809], Loss: 2.0886, Perplexity: 8.073355
Epoch [3/3], Step [500/809], Loss: 2.1439, Perplexity: 8.53254
Epoch [3/3], Step [600/809], Loss: 2.0478, Perplexity: 7.75057
Epoch [3/3], Step [700/809], Loss: 2.7415, Perplexity: 15.5098
Epoch [3/3], Step [800/809], Loss: 2.2179, Perplexity: 9.18822
Epoch [3/3], Step [809/809], Loss: 2.0102, Perplexity: 7.46505

Epoch time: 7852.449120521545




In [8]:
from workspace_utils import active_session  

start_epoch = 4
num_epochs = 4

with active_session():
    train(start_epoch, num_epochs)

Epoch [4/4], Step [100/809], Loss: 2.0705, Perplexity: 7.9291
Epoch [4/4], Step [200/809], Loss: 1.9414, Perplexity: 6.96834
Epoch [4/4], Step [300/809], Loss: 1.9689, Perplexity: 7.16281
Epoch [4/4], Step [400/809], Loss: 1.9541, Perplexity: 7.05734
Epoch [4/4], Step [500/809], Loss: 2.0280, Perplexity: 7.59892
Epoch [4/4], Step [600/809], Loss: 1.8779, Perplexity: 6.54009
Epoch [4/4], Step [700/809], Loss: 2.0029, Perplexity: 7.41048
Epoch [4/4], Step [800/809], Loss: 1.8996, Perplexity: 6.68331
Epoch [4/4], Step [809/809], Loss: 2.2638, Perplexity: 9.61972

Epoch time: 7868.33460688591




In [5]:
from workspace_utils import active_session  

start_epoch = 5
num_epochs = 5

with active_session():
    train(start_epoch, num_epochs)

Epoch [5/5], Step [100/809], Loss: 1.8015, Perplexity: 6.0589
Epoch [5/5], Step [200/809], Loss: 2.0160, Perplexity: 7.50831
Epoch [5/5], Step [300/809], Loss: 1.8344, Perplexity: 6.26126
Epoch [5/5], Step [400/809], Loss: 1.9688, Perplexity: 7.16239
Epoch [5/5], Step [500/809], Loss: 1.8574, Perplexity: 6.40684
Epoch [5/5], Step [600/809], Loss: 2.0136, Perplexity: 7.49017
Epoch [5/5], Step [700/809], Loss: 1.8944, Perplexity: 6.64862
Epoch [5/5], Step [800/809], Loss: 1.8898, Perplexity: 6.61773
Epoch [5/5], Step [809/809], Loss: 1.8351, Perplexity: 6.2656

Epoch time: 8011.097095966339




<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [1]:
# (Optional) TODO: Validate your model.

import torch
from torchvision import transforms
import os
import time
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from model import EncoderCNN, DecoderRNN
import data_loader_val


# False means the ground_truth is already there in a saved file; no need to generate it
generate_gt = True                                   

# TODO #1: Define a transform to pre-process the validation images.
# typically, we don't use any data augmentation during validation/testing
transform_val = transforms.Compose([
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    # transforms.RandomHorizontalFlip(),             # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Create a data loader for validation data
val_data_loader = data_loader_val.get_loader(transform=transform_val,
                         mode='val',
                         generate_gt=generate_gt)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!


In [2]:
encoder_file = "encoder-5.pkl"
decoder_file = "decoder-5.pkl" 

embed_size = 256
hidden_size = 1024

vocab_size = len(val_data_loader.dataset.vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=2)
decoder.eval()

# Load the trained weights
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

# Move models to GPU if CUDA is available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 99653985.18it/s]


DecoderRNN(
  (embed): Embedding(8855, 256)
  (lstm): LSTM(256, 1024, num_layers=2, batch_first=True)
  (fc): Linear(in_features=1024, out_features=8855, bias=True)
)

In [3]:
# same as that in notebook "3. Inference"
def clean_sentence(output):
    sentence = ""
    for i in range(len(output)):
        word = val_data_loader.dataset.vocab.idx2word[output[i]]
        if word == val_data_loader.dataset.vocab.start_word:
            continue
        if word == val_data_loader.dataset.vocab.end_word:
            break
        
        if word == '.':
            sentence = sentence[:-1] + word
        else:
            sentence = sentence + word + " "
    return sentence

In [4]:
cnt = 0            # used for logging validation progress
sample_size = 5000 # log after every sample_size items

pred_captions = []
ground_truth_captions = []


start_time = sample_start_time = time.time()

# iterate over validation data one image at a time (there are no batches)
for orig_image, image, img_id, annotations in iter(val_data_loader):
    # validation data loader returns original image, transformed image, image id, and ground-truth captions
    # original image was returned only for debugging purpose, so below code is not using it
    cnt += 1
    img_id = img_id.item()
    image = image.to(device)
    
    # predict caption for the image
    features = encoder(image).unsqueeze(1)
    output = decoder.sample(features)
    sentence = clean_sentence(output)
    
    # put predicted caption for the image in a list
    pred_captions.append({"image_id": img_id, "caption": [sentence]})
    
    # generate_gt (generate ground-truth) - True means we need to create ground-truth; False means we have already created it,
    # and it is stored in a file
    if generate_gt:
        for i in range(len(annotations)):
            # for some reason, while returning annotations from val_iter, one additional dimension gets added
            annotations[i] = annotations[i][0]
        # put ground-truth captions for the image in a list
        ground_truth_captions.append({"image_id": img_id, "caption":annotations})
    
    if cnt%sample_size == 0:
        print(f"Completed: {cnt}; time {time.time()-sample_start_time}")
        sample_start_time = time.time()
        
    # because of lack of GPU time, decided to use only 50k out of 200k validation samples
    if cnt == 50000:
        break

print(f"\n\nTotal time for processing validation data: {time.time()-start_time}\n\n")

Completed: 5000; time 276.8711516857147
Completed: 10000; time 275.643146276474
Completed: 15000; time 276.50507044792175
Completed: 20000; time 276.6227037906647
Completed: 25000; time 275.9075996875763
Completed: 30000; time 276.1495244503021
Completed: 35000; time 278.23764753341675
Completed: 40000; time 281.5620048046112
Completed: 45000; time 279.73266887664795
Completed: 50000; time 279.7780747413635


Total time for processing validation data: 2777.0121281147003




In [5]:
import json

# store the ground-truth and predicted captions in json files

if not generate_gt: # ground-truth already exists; read from file
    with open("gt_val_captions.json") as f:
        ground_truth_captions = json.load(f)
        print("Read groundtruth captions from file")
else: # ground-truth doesn't exist; save in a file for future runs
    with open("gt_val_captions.json", "w") as f:
        json.dump(ground_truth_captions, f)
        print("Saved groundtruth captions to file")

with open("pred_val_captions.json", "w") as f:
    json.dump(pred_captions, f)
    print("Saved predicted captions to file")

Saved groundtruth captions to file
Saved predicted captions to file


In [6]:
# convert list to dictionary - required for BLEU score computation code
# initially, they were stored in lists so that we could dump them in json files
pred_captions_dict = {}
ground_truth_captions_dict = {}

for i in range(len(pred_captions)):
    pred_captions_dict[pred_captions[i]['image_id']] = pred_captions[i]['caption']
    ground_truth_captions_dict[ground_truth_captions[i]['image_id']] = ground_truth_captions[i]['caption']

In [7]:
print(len(pred_captions))
print(len(pred_captions_dict))

50000
10082


In [8]:
# code taken from https://github.com/tylin/coco-caption/tree/master/pycocoevalcap/bleu
from bleu.bleu import Bleu

# compute BLEU score
val_bleu = Bleu()
score, scores = val_bleu.compute_score(ground_truth_captions_dict, pred_captions_dict)
print(score)
# print(scores) # gives some error because of large data

{'testlen': 92464, 'reflen': 92972, 'guess': [92464, 82382, 72300, 62218], 'correct': [54858, 23864, 8827, 3448]}
ratio: 0.9945359893301101
[0.5900397442154579, 0.4122902590982233, 0.274303690206387, 0.1836496197876687]


*Because of limited number of GPU hours, I ran validation code only on 50k items in Udacity workspace. Later, I moved the checkpoints from Udacity workspace to Google Colab and ran validation code on the whole dataset. The results are as below*

#### BLEU Scores (50k samples - Udacity Workspace)
Epoch 3: <br>
{'testlen': 97904, 'reflen': 96747, 'guess': [97904, 87822, 77740, 67658], 'correct': [55721, 23300, 8259, 3122]} <br>
ratio: 1.0119590271532863 <br>
[0.5691391567249492, 0.38858455245192486, 0.25220362889053277, 0.16494605543798443]


Epoch 4: <br>
{'testlen': 101049, 'reflen': 99150, 'guess': [101049, 90967, 80885, 70803], 'correct': [57871, 24268, 8867, 3352]} <br>
ratio: 1.0191527987897022 <br>
[0.5727023523241143, 0.39087648160283406, 0.2558561927827433, 0.16780698615782963]


Epoch 5: <br>
{'testlen': 92464, 'reflen': 92972, 'guess': [92464, 82382, 72300, 62218], 'correct': [54858, 23864, 8827, 3448]} <br>
ratio: 0.9945359893301101 <br>
[0.5900397442154579, 0.4122902590982233, 0.274303690206387, 0.1836496197876687]



#### BLEU Scores (whole dataset - Google Colab)

Epoch 3: <br>
{'testlen': 396813, 'reflen': 391116, 'guess': [396813, 356309, 315805, 275301], 'correct': [227481, 95304, 34458, 13327]} <br>
ratio: 1.0145660111066768 <br>
[0.5732700289557031, 0.391581158363211, 0.25576329176667817, 0.16869795260277565]

Epoch 4: <br>
{'testlen': 406064, 'reflen': 398221, 'guess': [406064, 365560, 325056, 284552], 'correct': [237056, 99201, 37296, 14920]} <br>
ratio: 1.0196950939302523 <br>
[0.5837897474289753, 0.39802185783953753, 0.2629294934985998, 0.17570384837107386]

Epoch 5: <br>
{'testlen': 374409, 'reflen': 375651, 'guess': [374409, 333905, 293401, 252897], 'correct': [226026, 96081, 35165, 13653]} <br>
ratio: 0.9966937396679338 <br>
[0.6016881604243738, 0.4154059397712154, 0.27418977855429866, 0.18249433404350074]